In [41]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import MinMaxScaler
from nltk.stem import PorterStemmer
from wordcloud import WordCloud, STOPWORDS
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk.probability import FreqDist
from nltk.corpus import stopwords

CARGA DE DATA

In [23]:
url='https://drive.google.com/file/d/1QuvhMiZLka18ZXnx8o1P5C8Cf5oHCgjL/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_labs = pd.read_csv(url)

In [24]:
drop_columns = ['original_language', 'runtime', 'spoken_languages', 'status', 
                'belongs_to_collection_name', 'production_companies', 'production_countries',
                'release_date']
df_labs.drop(columns=drop_columns, axis=1, inplace=True)


In [25]:
df_labs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45345 entries, 0 to 45344
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   budget        45345 non-null  float64
 1   id            45345 non-null  int64  
 2   overview      44404 non-null  object 
 3   popularity    45345 non-null  float64
 4   revenue       45345 non-null  float64
 5   title         45345 non-null  object 
 6   vote_average  45345 non-null  float64
 7   vote_count    45345 non-null  float64
 8   return        45345 non-null  float64
 9   genres        45345 non-null  object 
 10  release_year  45345 non-null  int64  
 11  cast          45345 non-null  object 
 12  director      45345 non-null  object 
dtypes: float64(6), int64(2), object(5)
memory usage: 4.5+ MB


In [ ]:
#Limpiar datos, crear caracteristicas y combinar colunmas
df_labs['genres'] = df_labs['genres'].fillna('')
df_labs['features'] = df_labs['genres'].astype(str) + ' ' + df_labs['vote_average'].astype(str)

In [ ]:
# Crear a TF-IDF matrix para caracteristicas combinadas
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df_labs['features'])

In [ ]:
#Definiendo la función de Recomendación
def recomendacion(titulo):
    titulo = titulo.lower()
    pelicula_seleccionada = df_labs[df_labs['title'].str.lower() == titulo]

    if pelicula_seleccionada.empty:
        return []

In [ ]:
 #Filtrar peliculas por titulo   
    index = pelicula_seleccionada.index[0]

In [ ]:
#Obtener el ID de la pelicula seleccionada
    id_pelicula_seleccionada = pelicula_seleccionada.iloc
# calcular la similitud entre peliculas 
    similarity_scores = cosine_similarity(tfidf_matrix[index], tfidf_matrix)
# Ranking de peliculas parecidas
    similar_movies_indices = similarity_scores.argsort()[0][::-1]
    similar_movies_indices = similar_movies_indices[similar_movies_indices != index]
    peliculas_similares = df_labs.iloc[similar_movies_indices][:5][['title', 'vote_average']].values.tolist()
#Peliculas con alta valoración
    peliculas_similares = sorted(peliculas_similares, key=lambda x: x[1], reverse=True)
    while len(peliculas_similares) < 5:
        peliculas_similares.append(['Película adicional', 0])

    return peliculas_similares

In [ ]:
#Crear una función para recomendar películas
@app.get("/recomendacion/{titulo}")
async def obtener_recomendacion(titulo: str):
    recomendaciones = recomendacion(titulo)
    if len(recomendaciones) == 0:
        return {"Error": "Película no encontrada"}
    else:
        return {"lista_recomendada": recomendaciones}

Stemming

In [ ]:
stemmer = PorterStemmer()